<a href="https://colab.research.google.com/github/IgnatiusEzeani/spatio-textual-colab-demos/blob/main/demo_2_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Classification with `spatio-textual`

In this demo, we explore the sentiment classification and analysis features withi the `spatio-textual` package.

It defaults to the a rule-based approach but includes the supports for large language models and HuggingFace

## Table of Contents
1. [Setup](#setup)
2. [Imports](#imports)
3. [Data & Quick Demo](#data-demo)
4. [Main Tutorial](#main)
5. [Tips & Troubleshooting](#tips)
6. [Summary](#summary)


## Setting up  <a id='setup'></a>

As earlier, download the `spaCy` model and install the `spatio-textual` package

In [ ]:
!python -m spacy download en_core_web_trf
!pip install -q git+https://github.com/SpaceTimeNarratives/spatio-textual.git

## Imports  <a id='imports'></a>

Let's import the necessary modules: `load_spacy_model` and `Annotator` from `spatio_textual.utils`; and `SentimentAnalyzer` from `spatio_textual.sentiment`

We also need `pandas` for working with data frames.

In [ ]:
import spatio_textual
from spatio_textual.utils import load_spacy_model, Annotator
from spatio_textual.sentiment import SentimentAnalyzer
import pandas as pd

## Data & Quick Demo  <a id='data-demo'></a>

In [ ]:
texts = [
    "I felt safe and relieved when we reached the farmhouse.",
    "We were afraid, hungry, and cold during the march.",
    "They asked us questions.",
]
sa = SentimentAnalyzer("rule")
sa.predict(texts)


## Main Tutorial  <a id='main'></a>
### 1) Annotate + attach sentiment

In [ ]:
nlp = load_spacy_model("en_core_web_sm")
ann = Annotator(nlp)
recs = ann.annotate_texts(texts, file_id="sent_demo", include_text=True)
preds = sa.predict([r["text"] for r in recs])
for r, p in zip(recs, preds):
    r.update({"sentiment_label": p["label"], "sentiment_score": p["score"]})
pd.DataFrame([{k:r.get(k) for k in ["segId","text","sentiment_label","sentiment_score"]} for r in recs])


### 2) Hooking up an HF pipeline (example)

In [ ]:
# from transformers import pipeline
# hf = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")
# hf(texts)


## Tips & Troubleshooting  <a id='tips'></a>
- Rule backend is offline and immediate but simplistic; HF/LLM provide richer signals.
- Keep inputs as short segments for better classifier performance.


## Summary  <a id='summary'></a>
You ran sentiment classification with the rule backend and saw how to plug an HF pipeline.
